# Data Understanding of Teacher dataset

(This document relates to question number 2)

- What pedagogical strategies developed by teachers seem to promote better reading performance (PISA 2018 - teacher context data)?
- What pedagogical strategies, according to the students' perspective, seem to promote better reading performance (PISA 2018 - student context data)?
- Do teachers and students have the same perceptions?

## **2.1 Collect Initial Data**

For this question we're using 2 datasets (https://www.oecd.org/en/data/datasets/pisa-2022-database.html):
- Students database 2018 (PISA)
- Teachers database 2018 (PISA) (In this file we use only the teacher database)

The source datasets are in .sas7bdat format, that we converted to .csv with the following command:

In [2]:
import pandas as pd

"""
data = pd.read_sas(
    "../../../databases/2018/cy07_msu_stu_qqq.sas7bdat", format="sas7bdat"
)

data.to_csv("../../../databases/2018/student2018.csv", index=False)

data = pd.read_sas(
    "../../../databases/2018/cy07_msu_tch_qqq.sas7bdat", format="sas7bdat"
)

data.to_csv("../../../databases/2018/teacher2018.csv", index=False)
"""

'\ndata = pd.read_sas(\n    "../../../databases/2018/cy07_msu_stu_qqq.sas7bdat", format="sas7bdat"\n)\n\ndata.to_csv("../../../databases/2018/student2018.csv", index=False)\n\ndata = pd.read_sas(\n    "../../../databases/2018/cy07_msu_tch_qqq.sas7bdat", format="sas7bdat"\n)\n\ndata.to_csv("../../../databases/2018/teacher2018.csv", index=False)\n'

In [3]:
teacher = pd.read_csv('../../../databases/2018/teacher2018.csv', nrows=1000)
#teacher = pd.read_csv('../../../databases/2018/teacher2018.csv')

**Note:** We don't include these files in the project folder, so it's necessary to manually download and put them in their respective folder.

## **2.2 Describe Data**

#### Teacher Data

The original dataset has 350 features

In [4]:
teacher.head()

,CNTRYID,CNT,CNTSCHID,CNTTCHID,TEACHERID,CYC,NatCen,Region,STRATUM,SUBNATIO,...,FEEDBACK,ADAPTINSTR,FEEDBINSTR,TCATTIMM,GCTRAIN,TCMCEG,GCSELF,W_SCHGRNRABWT,W_FSTUWT_SCH_SUM,VER_DAT
0,8.0,b'ALB',800057.0,800001.0,5.0,b'07MS',b'000800',800.0,b'ALB0203',b'0080000',...,0.6122,NaN,NaN,1.2332,1.2735,-0.1151,1.9399,3.45567,93.30319,b' 09MAY19:11:21:10'
1,8.0,b'ALB',800121.0,800002.0,5.0,b'07MS',b'000800',800.0,b'ALB0107',b'0080000',...,-1.2284,NaN,NaN,-0.8666,1.8008,-0.1151,-0.2634,6.07334,12.46632,b' 09MAY19:11:21:10'
2,8.0,b'ALB',800140.0,800003.0,5.0,b'07MS',b'000800',800.0,b'ALB0101',b'0080000',...,0.2741,NaN,NaN,0.4987,1.8008,0.5387,-0.7038,3.76885,99.20152,b' 09MAY19:11:21:10'
3,8.0,b'ALB',800149.0,800004.0,5.0,b'07MS',b'000800',800.0,b'ALB0211',b'0080000',...,-0.2922,NaN,NaN,1.2332,0.2917,-0.6041,-1.0056,6.62948,19.88845,b' 09MAY19:11:21:10'
4,8.0,b'ALB',800095.0,800005.0,5.0,b'07MS',b'000800',800.0,b'ALB0204',b'0080000',...,0.1202,NaN,NaN,0.4987,1.0055,-1.9057,1.9399,3.35644,104.76897,b' 09MAY19:11:21:10'


In [84]:
teacher.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107367 entries, 0 to 107366
Data columns (total 350 columns):
 #    Column            Non-Null Count   Dtype  
---   ------            --------------   -----  
 0    CNTRYID           107367 non-null  float64
 1    CNT               107367 non-null  object 
 2    CNTSCHID          107367 non-null  float64
 3    CNTTCHID          107367 non-null  float64
 4    TEACHERID         91190 non-null   float64
 5    CYC               107367 non-null  object 
 6    NatCen            107367 non-null  object 
 7    Region            107367 non-null  float64
 8    STRATUM           107367 non-null  object 
 9    SUBNATIO          107367 non-null  object 
 10   OECD              107367 non-null  float64
 11   ADMINMODE         107367 non-null  float64
 12   LANGTEST          93006 non-null   float64
 13   TC001Q01NA        90090 non-null   float64
 14   TC002Q01NA        90326 non-null   float64
 15   TC005Q01NA        90408 non-null   float64
 16   

In [5]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../../../src')) # Add the src directory to the Python path

from pisadatamap.pisadatamap import PISADataMap

data_map = PISADataMap('../../../databases/2018/teacher_data_structure_2018.csv')

for column in data_map.map_enum:
    print(f"\033[1m{column.name}\033[0m: {column.value}")

CNTRYID: Country Identifier
CNT: Country code 3-character
CNTSCHID: Intl. School ID
CNTTCHID: Intl. Teacher ID
TEACHERID: Teacher identification code
CYC: PISA Assessment Cycle (2 digits + 2 character Assessment type - MS/FT)
NatCen: National Centre 6-digit Code
Region: Region
STRATUM: Stratum ID 7-character (cnt + region ID + original stratum ID)
SUBNATIO: Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)
OECD: OECD country
ADMINMODE: Mode of Respondent
LANGTEST: Language of Questionnaire/Assessment
TC001Q01NA: Are you female or male?
TC002Q01NA: How old are you?
TC005Q01NA: What is your current employment status as a teacher? My employment status at this school
TC007Q01NA: How many years of work experience do you have? Year(s) working as a teacher at this school
TC007Q02NA: How many years of work experience do you have? Year(s) working as a teacher in total
TC014Q01HA: Did you complete a teacher education or training programme?
TC015Q01NA: How d

C:\Users\david\Documents\iscte\2 semestre\[PIAA] Projeto de Inteligência Artificial Aplicada\piaa\src\pisadatamap\pisadatamap.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.map_enum = Enum('MapEnum', {row[0]: row[1] for index, row in map_df.iterrows()}) # Setup enum with key, values of the codebook


The dataset is composed by 343 numeric columns and only 7 categorical columns.

In [6]:
import pandas as pd
from tabulate import tabulate

categorical_columns = teacher.select_dtypes(include=["object", "category"]).columns
numeric_columns = teacher.select_dtypes(include=["int64", "float64"]).columns

column_types_df = pd.DataFrame(
    {
        "Column type": ["Numeric", "Categorical"],
        "Number of columns": [len(numeric_columns), len(categorical_columns) ],
        "Column names": [
            ", ".join(numeric_columns),
            ", ".join(categorical_columns),
        ],
    }
)

print(
    tabulate(
        column_types_df,
        headers="keys",
        tablefmt="pretty",
        showindex=False,
        colalign=("left", "left", "left"),
    )
)

+-------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
teacher.describe()

,CNTRYID,CNTSCHID,CNTTCHID,TEACHERID,Region,OECD,ADMINMODE,LANGTEST,TC001Q01NA,TC002Q01NA,...,TCDIRINS,FEEDBACK,ADAPTINSTR,FEEDBINSTR,TCATTIMM,GCTRAIN,TCMCEG,GCSELF,W_SCHGRNRABWT,W_FSTUWT_SCH_SUM
count,1000.0,1000.000000,1000.000000,972.000000,1000.0,1000.0,1000.0,972.0,962.000000,963.000000,...,146.000000,803.000000,146.000000,146.000000,802.000000,768.000000,804.000000,803.000000,1000.000000,1000.000000
mean,8.0,800171.222000,800513.832000,4.829218,800.0,0.0,2.0,140.0,1.284823,42.091381,...,0.692434,0.463599,0.595611,0.475650,0.690982,0.746902,0.073679,0.874636,3.992718,104.024420
std,0.0,93.680269,295.948883,0.376512,0.0,0.0,0.0,0.0,0.451565,10.608315,...,0.902041,0.913956,0.922965,0.790719,0.765817,0.935982,1.091519,1.024820,2.895212,64.165826
min,8.0,800002.000000,800001.000000,4.000000,800.0,0.0,2.0,140.0,1.000000,20.000000,...,-3.926200,-2.998400,-1.670600,-1.367100,-1.694000,-1.337500,-2.716000,-1.667400,1.000000,1.116790
25%,8.0,800092.000000,800257.750000,5.000000,800.0,0.0,2.0,140.0,1.000000,34.000000,...,0.391800,-0.284400,0.099500,-0.104500,0.258700,0.115600,-0.626600,-0.263400,1.422867,53.478533
50%,8.0,800172.000000,800513.500000,5.000000,800.0,0.0,2.0,140.0,1.000000,41.000000,...,1.046800,0.529700,0.480400,0.502750,1.233200,0.718900,0.420300,1.156300,3.439850,96.203030
75%,8.0,800252.000000,800770.250000,5.000000,800.0,0.0,2.0,140.0,2.000000,50.000000,...,1.359600,1.093650,1.109000,0.990900,1.233200,1.800800,1.125000,1.939900,6.073340,146.875957
max,8.0,800336.000000,801026.000000,5.000000,800.0,0.0,2.0,140.0,2.000000,69.000000,...,1.359600,1.827700,2.453300,1.597500,1.297200,1.800800,1.125000,1.939900,15.075400,361.809600


## 2.4 Verify data quality


In [89]:
print("\n--- Missing Values ---")
missing = teacher.isnull().mean().sort_values(ascending=False)
print(missing[missing > 0.7])

Resumo de valores ausentes:

            Missing Values  Percentage Missing
EC162Q08HA          577987               94.44
EC162Q06HA          576013               94.12
EC162Q07HA          575891               94.10
EC162Q04HA          575680               94.06
EC162Q05HA          575435               94.02
...                    ...                 ...
PV7READ               5377                0.88
PV6READ               5377                0.88
PV3READ               5377                0.88
GRADE                 3019                0.49
ST004D01T                2                0.00

[1007 rows x 2 columns]

Número de linhas com valores ausentes: 612004

Exemplo de linhas com valores ausentes:
   CNTRYID     CNT  CNTSCHID  CNTSTUID      CYC     NatCen     STRATUM  \
0      8.0  b'ALB'  800115.0  800001.0  b'07MS'  b'000800'  b'ALB0107'   
1      8.0  b'ALB'  800300.0  800002.0  b'07MS'  b'000800'  b'ALB0105'   
2      8.0  b'ALB'  800088.0  800003.0  b'07MS'  b'000800'  b'ALB0101'  